In [ ]:
%load_ext autoreload
%autoreload 2
from IPython.display import display
from notion_client import Client
from pprint import pprint
import xml.etree.ElementTree as ET

from notion_api import *
from _secrets import NOTION_API_KEY
from wordpress_xml import extract_posts, Posts, write_posts

WORDPRESS_XML_FILEN = "/Users/greg/Desktop/gregdetre.com www website/gregdetre.wordpress.com-2024-08-16-13_30_40/gregdetre.wordpress.2024-08-16.000.xml"
# WORDPRESS_XML_FILEN = "/Users/greg/Desktop/gregdetre.com www website/gregdetre.wordpress.com-2024-08-16-13_30_40/truncated.xml"
WRITE_DIRN = "/Users/greg/Desktop/gregdetre.com www website/240818_1700 exported html and json"
NOTION_DATABASE_URL = 'https://www.notion.so/glow-flavor-ed2b/8866d83090bf43e79e61c42aaa6a8639?v=03507c8a53f04221b6f733fd990c2615'

tree = ET.parse(WORDPRESS_XML_FILEN)

notion = Client(auth=NOTION_API_KEY)

In [ ]:
wp_posts = extract_posts(tree, verbose=0)

In [ ]:
html_filens, json_filens = write_posts(wp_posts, WRITE_DIRN)

In [ ]:
# page_id = get_id_from_url(
#     # 'https://www.notion.so/glow-flavor-ed2b/Import-Aug-18-2024-part-1-a4f4f11c93ca4f94bc4787765f70771a'
#     'https://www.notion.so/glow-flavor-ed2b/8866d83090bf43e79e61c42aaa6a8639?v=03507c8a53f04221b6f733fd990c2615'
#     )
# page = notion.pages.retrieve(page_id=page_id)
# # page = notion.pages.retrieve(page_id='27b05f38fcff46459e432d9c3a3f0069')
# # page

In [ ]:
# notion_title_from_id = get_child_pages(notion, page_id)
# len(notion_title_from_id)
pages = get_database_pages(notion, NOTION_DATABASE_URL)
print(f"{len(pages)=}")
notion_title_from_id = {page['id']: page['properties']['Name']['title'][0]['text']['content'] for page in pages}
# notion_title_from_id

In [ ]:
def match_wp_posts_notion_ids(
    notion: Client, wp_posts: Posts, notion_title_from_id: dict[str, str], verbose: int = 0
):
    notion_id_from_title = {no_title: no_id for no_id, no_title in notion_title_from_id.items()}
    assert len(notion_id_from_title) == len(notion_title_from_id)
    no_id_from_wp_id = {}
    for post_id, post_data in wp_posts.items():
        wp_id = post_id
        wp_title = post_data["title"]
        no_id = notion_id_from_title.get(wp_title)
        if no_id is None:
            if verbose >= 1:
                print(f"Skipping {wp_id} {wp_title}")
            continue
        if verbose >= 2:
            print("Found:", no_id, wp_id, wp_title)    
        no_id_from_wp_id[wp_id] = no_id
    return no_id_from_wp_id

no_id_from_wp_id = match_wp_posts_notion_ids(notion, wp_posts, notion_title_from_id, verbose=1)

In [ ]:
def update_notion_properties_from_wordpress(
    notion: Client, wp_posts: Posts, no_id_from_wp_id: dict[str, str], verbose: int = 0
):
    def package_text(txt: Optional[str]):
        if txt is None:
            txt = ''
        return {"rich_text": [{"type": "text", "text": {"content":txt}}]}
    
    def package_date(dt: str):
        return {"date": {"start": dt}}
    
    def package_multi_select(items: list[str]):
        return {"multi_select": [{"name": x} for x in items]}

    for wp_id, no_id in no_id_from_wp_id.items():
        # print(wp_id)
        post_data = dict(wp_posts[wp_id]) # so that it's a copy
        no_title = post_data["title"]
        # e.g. "Tags": {"multi_select": [{"name": "Tag1"}, {"name": "Tag2"}]},
        del post_data["title"]
        del post_data["content"]
        post_data['categories'] = package_multi_select(post_data['categories'])
        post_data['tags'] = package_multi_select(post_data['tags'])
        post_data['post_date'] = package_date(post_data['post_date'])
        post_data['post_modified'] = package_date(post_data['post_modified'])
        post_data['status'] = package_text(post_data['status'])
        post_data['excerpt'] = package_text(post_data['excerpt'])
        post_data['post_type'] = package_text(post_data['post_type'])
        post_data['slug'] = package_text(post_data['slug'])

        try:
            notion.pages.update(
                page_id=no_id,
                properties=post_data
            )
            if verbose >= 2:
                print(f"Updated {no_id} {no_title}")
        except:
            if verbose >= 1:
                print(f"Failed {no_id} {no_title}")
            # raise
    
update_notion_properties_from_wordpress(notion, wp_posts, no_id_from_wp_id, verbose=2)